# Project 1B - Modelling Sparkify Data in Cassandra

In this project we are given an event log stream from Sparkify, and 3 analytics queries which the Sparkify team wish to run against this data. After preprocessing the raw data, we take the approach of designing a Cassandra table per query, populating these tables and finally running test queries to verify the system is working correctly.  

# Data Pre-Processing

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# show the total number of rows retrieved 
len(full_data_rows_list)

8056

In [4]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Modelling the Data in Cassandra

Our raw data has the following columns:

0. artist 
1. firstName of user
2. gender of user
3. item number in session
4. last name of user
5. length of the song
6. level (paid or free song)
7. location of the user
8. sessionId
9. song title
10. userId


<img src="images/image_event_datafile_new.jpg">

We will create and populate one table per query which the Sparkify team wish to make.

First, we initialize a new database connection and Keyspace

#### Creating a Cluster on localhost

In [6]:
from cassandra.cluster import Cluster
cluster = Cluster()

session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Query 1) Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

Or in CQL:

```
select artist, song_title, song_length from app_history where session_id = 338 and item_in_session = 4
```

We partition on `session_id` and cluster on `item_in_session`, giving unique entries for each (`session_id`,`item_in_session`) pair, and accommodating the where clause. Creating the table:

In [9]:
query = "CREATE TABLE IF NOT EXISTS app_history_by_sessions "
query = query + "(session_id int, item_in_session int, artist text, song_title text, song_length float, PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

Inserting data from our preprocessed csv file:

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_history_by_sessions (session_id, item_in_session, artist, song_title, song_length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

Finally, we test by running the query

In [11]:
query = "SELECT artist, song_title, song_length FROM app_history_by_sessions WHERE session_id = 338 and item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

pd.DataFrame(list(rows))

,artist,song_title,song_length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Query 2) Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

```
select artist, song_title, first_name, last_name from app_history where user_id = 10 and session_id = 182 order by item_in_session asc
```

Partition on `user_id` and cluster on `session_id`. To order by `item_in_session` we add it as a cluster key.


In [12]:
query = "CREATE TABLE IF NOT EXISTS app_history_by_users "
query = query + "(user_id int, session_id int, item_in_session int, artist text, song_title text, first_name text, last_name text, PRIMARY KEY (user_id, session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)

Inserting data from our preprocessed csv file:

In [13]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_history_by_users (user_id, session_id, item_in_session, artist, song_title, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

Testing with the provided query:

In [14]:
query = "SELECT artist, song_title, first_name, last_name from app_history_by_users where user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

pd.DataFrame(list(rows))

,artist,song_title,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


Let's also check that songs are correctly ordered by selecting `item_in_session`

In [15]:
query = "SELECT artist, song_title, item_in_session, first_name, last_name from app_history_by_users where user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

pd.DataFrame(list(rows))

,artist,song_title,item_in_session,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,0,Sylvie,Cruz
1,Three Drives,Greece 2000,1,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,2,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,3,Sylvie,Cruz


### Query 3) Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

```
# select first_name, last_name from app_history_by_songs where song_title = 'All Hands Against His Own'
```

For this query, we're interested in only unique (user, song_title) pairs, since we do not need to record repeated plays of the same song by the same user. Thus we can user the primary key `(song_title, user_id)`.


In [16]:
query = "CREATE TABLE IF NOT EXISTS app_history_by_songs "
query = query + "(song_title text, user_id int, first_name text, last_name text, PRIMARY KEY (song_title, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)

Inserting data from our preprocessed csv file:

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO app_history_by_songs (song_title, user_id, first_name, last_name) "
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))              

Test with a query:

In [18]:
query = "SELECT first_name, last_name from app_history_by_songs where song_title = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

pd.DataFrame(list(rows))  

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


# Session Cleanup

#### Drop the tables before closing out the sessions

In [19]:
query = "drop table app_history_by_sessions"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table app_history_by_users"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table app_history_by_songs"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()